In [1]:
from glob import glob
from typing import Tuple, List
from pathlib import Path
import os.path as osp
import cv2
import albumentations as A
from fl_tissue_model_tools.transforms import get_elastic_dual_transform
from fl_tissue_model_tools.preprocessing import get_augmentor, augment_images
from fl_tissue_model_tools.helper import get_img_mask_paths

In [2]:
'''
This notebook is used to test the augmentation pipeline for binary segmentation.
'''

albumentations_transform = A.Compose([
    A.Rotate(p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0),
    A.RandomCrop(height=572, width=572),
    A.Flip(p=0.5),
    A.OneOf([
            A.MultiplicativeNoise(p=0.5),
            A.AdvancedBlur(p=0.5)
    ], p=0.8),
    A.OneOf([
        A.RandomGamma(p=0.2),
        A.RandomBrightnessContrast(p=0.4),
        A.RandomToneCurve(scale=0.75, p=0.2),
        A.CLAHE(p=0.2)
    ], p=0.75)
])

elastic_transform = get_elastic_dual_transform()

transform_list = [albumentations_transform, elastic_transform]
augmentor = get_augmentor(transform_list)

In [8]:
[*zip([1,2,3], [4,5,6])]

[(1, 4), (2, 5), (3, 6)]

In [5]:
img_dir = '/home/bean/lab/training-data/jan31'
mask_dir = img_dir
out_path = "transforms_output"
out_ext = ".png"
num_cycles = 1
img_paths, mask_paths = get_img_mask_paths(img_dir, mask_dir)
images = [cv2.imread(fp, 0) for fp in img_paths]
masks = [cv2.imread(fp, 0) for fp in mask_paths]
im_mask_pairs = list(zip(images, masks))
Path(out_path).mkdir(parents=True, exist_ok=True)

In [6]:
for cycle in range(num_cycles):
	augmented = augment_images(images, masks, augmentor)
	cycle_num = str(cycle + 1) if num_cycles > 1 else ''
	for i, (img, mask) in enumerate(augmented):
		sample_name = Path(img_paths[i]).stem
		cv2.imwrite(f"{out_path}/{sample_name}_aug{cycle_num}{out_ext}", img)
		cv2.imwrite(f"{out_path}/{sample_name}_mask_aug{cycle_num}{out_ext}", mask)